# Register Pytorch Model in SAS Viya 4 Model Manager

In [8]:
import sasctl.pzmm as pzmm
import sasctl
from sasctl import Session
import pandas as pd
import getpass
import numpy
import os


In [5]:
# Define naming for the project and prefix for all the models
prefixModelFile = 'myProject_'
model_name = prefixModelFile + 'myModel'

In [1]:
# Define a directory where all the files for SAS Model Manager will be kept 
MODEL_DIR = '/python-sasctl/pytorch/'  # this cell clears all files inside MODEL_DIR path!!!


if not(os.path.exists(MODEL_DIR)):
    os.makedirs(MODEL_DIR)
    print("Directory " , MODEL_DIR ,  " created")
else:
    print("Directory " , MODEL_DIR,  " already exists")
    for filename in os.listdir(MODEL_DIR):
        file_path = os.path.join(MODEL_DIR, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
                print('older file deleted...')
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
                print('older file deleted...')
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

### Define Server Credentials

In [40]:
server = 'SERVER'
user = 'USERNAME'

print('Password for the Server:')
p = getpass.getpass()

Password for the Server:
········


### Define Input/output variables

In [10]:
# Input variables
inputVar = pd.DataFrame({'image' : ['msg'*33333]})
pzmm.JSONFiles.write_var_json(inputDF=inputVar,isInput=True)
inputVarJSON = pd.read_json("inputVar.json")

# Output variables
outputVar = pd.DataFrame({'food_label' : ['msg'*90], 'msg' : ['msg'*90]})
pzmm.JSONFiles.write_var_json(inputDF=outputVar,isInput=False)
outputVarJSON =  pd.read_json("outputVar.json")

# Define Input Output Variables and types for SAS Model Manager
project_input_variables = list(pd.DataFrame.to_dict(inputVarJSON.transpose()).values())
for var in project_input_variables:
    var["role"] = "input"
project_output_variables = list(pd.DataFrame.to_dict(outputVarJSON.transpose()).values())
for var in project_output_variables:
    var["role"] = "output"
project_variables = project_input_variables + project_output_variables

inputVar.json was successfully written and saved to Model/inputVar.json
outputVar.json was successfully written and saved to Model/outputVar.json


### Define Score Code for Pytorch Model

In [14]:
score_code = """
from torch import jit
import torch
from torchvision import transforms
import base64
from PIL import Image
import io
import settings

model = None

def scoreModel(image):
    "Output: food_label, msg"
    
    global model
    try:
        # load labels 
        with open(settings.pickle_path + "class_names.txt", "r") as file:
            labels = file.read()
        labels = labels.split(",")

        # transform image data
        data_transforms = transforms.Compose([transforms.Resize(256),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

        # read as string and decode base64
        my_byte = bytes(image, encoding="UTF-8")
        r = base64.decodebytes(my_byte)

        # convert into a numeric matrix
        image = Image.open(io.BytesIO(r)).convert("RGB")
        # transform into torch format
        image = data_transforms(image)
        
        if model is None:
            # load torch model
            model = jit.load(settings.pickle_path + "pytorch_model.pt", map_location=torch.device("cpu"))
        
        # predict the index
        _,preds = torch.max(model(image[None, ...]), 1)
        food_label = labels[preds[0]]
        msg = "success"
    except Exception as e:
        food_label = "-1"
        msg = str(e)
    return food_label, msg"""

f = open(MODEL_DIR + '/' + model_name + '.py',"w+")
f.write(score_code)
f.close()

### Add requirements.json File for SCR Container
Installs additional components inside the container

In [15]:
requirements = """[
     {
        "step":"install pandas ",
        "command":"pip3 install pandas"
     },
     {
        "step":"install base64 ",
        "command":"pip3 install pybase64"
     },
     {
        "step":"install Pillow ",
        "command":"pip3 install Pillow"
     },
     {
        "step":"install sasctl ",
        "command":"pip3 install sasctl"
     },
     {
        "step":"install pytorch ",
        "command":"pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu"
     }
]"""

f = open(MODEL_DIR + '/' + 'requirements.json',"w+")
f.write(requirements)
f.close()

In [39]:
model_attr = dict()
model_attr['name'] = model_name


model_attr['toolVersion'] = '3.8.12'
model_attr['eventProbVar'] = 'food_label'
model_attr['targetLevel'] = 'NOMINAL'
model_attr['trainCodeType'] = 'Python'


project_name = 'myProjectName'

### Create Project in SAS Model Manager

In [17]:
from sasctl import Session
from sasctl.services import model_repository as mr

try:
    # Establish a session with SAS Viya
    with Session(server, user, p, verify_ssl=False, protocol='http') as s:
        mr.create_project(project=project_name, repository = 'Public', variables = project_variables,targetLevel = model_attr['targetLevel'],function='classification')
except Exception as e:
    print(e)

### Edit Project Settings

In [18]:
with Session(server, user, p, verify_ssl=False, protocol = 'http'):

    project = mr.get_project(project_name)
    project['eventProbabilityVariable'] = model_attr['eventProbVar']
    project = mr.update_project(project)

### Create Model in SAS Model Manager

In [37]:
with Session(server, user, p, verify_ssl=False, protocol = 'http'):

    mr.create_model(model=model_attr,project=project_name, modeler= 'User1', function='classification', 
                    algorithm='Pytorch efficientnet', tool='Python 3', target_variable=model_attr['eventProbVar'],
                    score_code_type= "python",
                    input_variables=list(pd.DataFrame.to_dict(inputVarJSON.transpose()).values()),
                    output_variables=list(pd.DataFrame.to_dict(outputVarJSON.transpose()).values()),
                    is_champion=True)

### Upload Pytorch Model Assets

In [ ]:
# Establish a session with SAS Viya

with Session(server, user, p, verify_ssl=False, protocol = 'http'):
    # score script
    mr.add_model_content(model=model_name, file=score_code, 
                        name=model_name + '.py', role='score')

    # saved model
    file = open('data/pytorch/pytorch_model.pt', 'rb')
    mr.add_model_content(model=model_name, file=file, 
                         name='pytorch_model.pt', role='Python pickle')
    file.close()
    
    # requirements for SCR environment
    file = open(MODEL_DIR + '/' + 'requirements.json', 'rb')
    mr.add_model_content(model=model_name, file=file, 
                         name='requirements.json', role='python pickle')
    file.close()
    
    # saved list class names
    file = open('data/pytorch/class_names.txt', 'rb')
    mr.add_model_content(model=model_name, file=file, 
                         name='class_names.txt', role='Python pickle')
    file.close()

### Deploy Model

In [42]:
from sasctl.services import model_management as mm
# Establish a session with SAS Viya
with Session(server, user, p, verify_ssl=False, protocol = 'http') as s:
    # Publish the model to the azure registry with git repo
    module = mm.publish_model(model_name, destination='YOUR-CONTAINER-DESTINATION', force=True)

# Score Model using Local SCR

### Create Input Object

In [ ]:
import base64

with open("data/pytorch/Image_107.jpg", "rb") as img_file:
    my_string = base64.b64encode(img_file.read())
    image_strings = my_string.decode('utf-8')

In [ ]:
from io import StringIO
import csv


f = StringIO()
w = csv.writer(f)
w.writerow(['image'])
w.writerow([image_strings])
input_byte = f.getvalue().encode('UTF-8')

### Make API Request to a Local Container Listening on Port 8080

In [ ]:
import requests

# Run a test on score API
headers = {'Content-Type':"multipart/form-data"}
endpoint = "http://localhost:8080/executions"

files = {'file': ('score.csv', input_byte)}
response = requests.post(endpoint, files=files)


print('endpoint= ', endpoint)
print('http status code= ' , response.status_code)
print('elapsed time= ' , response.elapsed)
print('output data= \n' , response.json())


### Get Output.csv

In [ ]:
endpoint = f"http://localhost:8080/query/{response.json()['id']}"
response = requests.get(endpoint)

print('endpoint= ', endpoint)
print('http status code= ' , response.status_code)
print('elapsed time= ' , response.elapsed)

In [ ]:
from io import StringIO

TESTDATA = StringIO(response.text)
pd.read_csv(TESTDATA)